# Geocoding Data
After we have got data from Step 1, please procceed with making sure that there is one source of truth about location: different locations registered on Twitter users are in the most the same city or prefecture. The task is to union all of them to one.

In [4]:
import pandas as pd
from core.geocoder import process_locations_gmaps
from core.utils import convert_to_datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

df = convert_to_datetime(pd.read_excel('./src/data/2009_elections.xlsx'))
print(f'📊 {len(df)} tweets to process')

location_df = df[df['user_location'].notna()]
print(f'📊 {len(location_df)} not null locations tweets to process\n---')
print(f'📊 {len(location_df["user_location"].unique())} unique locations to process')

📅 Converted tweet_date to datetime
📊 103519 tweets to process
📊 81183 not null locations tweets to process
---
📊 9473 unique locations to process


In [ ]:
location_df['user_location'] = location_df['user_location'].str.strip()
location_df['user_location'] = location_df['user_location'].str.lower()
location_df['user_location'] = location_df['user_location'].str.replace('iphone: ', '')
location_df['user_location'] = location_df['user_location'].str.replace('iphone', '')
location_df['user_location'] = location_df['user_location'].str.replace('⇄', '')
location_df['user_location'] = location_df['user_location'].str.replace('⇔', '')
location_df['user_location'] = location_df['user_location'].str.replace('⇔', '')
location_df['user_location'] = location_df['user_location'].str.replace('→', '')
location_df['user_location'] = location_df['user_location'].str.strip()
location_df = location_df[location_df['user_location'].notna()]
len(location_df['user_location'].unique())

In [6]:
import json
geocoded_output = process_locations_gmaps(location_df) # encoding default convert to uft-8

to_export = json.loads(geocoded_output)
with open('./src/mappings/2009_loc_map_gmaps.json', 'w') as f:
    json.dump(to_export, f, indent=4)

 27%|██▋       | 2479/9225 [00:43<01:56, 57.96it/s]

An error occurred while geocoding '': HTTP Error: 400


100%|██████████| 9225/9225 [02:59<00:00, 51.34it/s]


AttributeError: 'str' object has no attribute 'items'

In [40]:
import json
import folium
from folium.plugins import MarkerCluster

# Load the JSON data
with open('./src/mappings/2009_loc_map_gmaps.json', 'r') as file:
    loaded = json.load(file)

print(f'📊 {len(loaded)} geocoded locations')

# Create a base map centered around Japan
japan_map = folium.Map(location=[36.2048, 138.2529], zoom_start=5)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(japan_map)

# Iterate through the loaded JSON data and add markers to the map
for location in loaded.values():
    if location is not None and 'geometry' in location and 'location' in location['geometry']:
        latitude = location['geometry']['location']['lat']
        longitude = location['geometry']['location']['lng']
        name = location['formatted_address']

        marker = folium.Marker([latitude, longitude], popup=name)
        marker.add_to(marker_cluster)
    else:
        None

# Save the map as an HTML file
japan_map.save("japan_map.html")


📊 9225 geocoded locations


In [45]:
import json
import folium
from folium.plugins import MarkerCluster

# Load the JSON data
with open('./src/mappings/2009_loc_map_gmaps.json', 'r') as file:
    loaded = json.load(file)

print(f'📊 {len(loaded)} geocoded locations')

# Create a base map centered around Japan
japan_map = folium.Map(location=[36.2048, 138.2529], zoom_start=5)

# Create a dictionary to store marker clusters per prefecture
prefecture_clusters = {}
prefecture_stats = {}


def get_prefecture(location_data):
    for component in location_data['address_components']:
        if 'administrative_area_level_1' in component['types']:
            return component['long_name']
    return None

# Iterate through the loaded JSON data and add markers to the map
for location in loaded.values():
    if location is not None and 'geometry' in location and 'location' in location['geometry']:
        latitude = location['geometry']['location']['lat']
        longitude = location['geometry']['location']['lng']
        name = location['formatted_address']

        # Extract the prefecture from the address_components
        prefecture = get_prefecture(location)
        if prefecture is None:
            continue
        
        if prefecture not in prefecture_stats:
            prefecture_stats[prefecture] = 0
        prefecture_stats[prefecture] += 1
        
        # If the prefecture does not already have a marker cluster, create one
        if prefecture not in prefecture_clusters:
            marker_cluster = MarkerCluster().add_to(japan_map)
            prefecture_clusters[prefecture] = marker_cluster

        marker = folium.Marker([latitude, longitude], popup=name)
        marker.add_to(prefecture_clusters[prefecture])
    else:
        None

# Print out the stats for each prefecture
print("Stats per Prefecture:")
for prefecture, count in prefecture_stats.items():
    print(f"{prefecture}: {count} locations")

japan_map.save("japan_map.html")

📊 9225 geocoded locations
Error extracting prefecture: Japan
Error extracting prefecture: Kanto Region, Japan
Error extracting prefecture: Japan
Error extracting prefecture: Pacific Ocean
Error extracting prefecture: Seto Inland Sea, Japan
Error extracting prefecture: Japan
Error extracting prefecture: Japan
Error extracting prefecture: New Zealand
Error extracting prefecture: Japan
Error extracting prefecture: Singapore
Error extracting prefecture: Taiwan
Error extracting prefecture: Japan
Error extracting prefecture: Kyushu, Japan
Error extracting prefecture: Japan
Error extracting prefecture: Kansai, Japan
Error extracting prefecture: Kansai, Japan
Error extracting prefecture: Japan
Error extracting prefecture: Tohoku Region, Japan
Error extracting prefecture: Japan
Error extracting prefecture: Kivotos 510 30, Greece
Error extracting prefecture: Japan
Error extracting prefecture: Japan
Error extracting prefecture: Kanto Region, Japan
Error extracting prefecture: Japan
Error extracti